In [71]:
#all of the codes are the step by step example of chapter 8 working with Databases
import sqlite3

with sqlite3.connect(':memory:') as con:
    c = con.cursor()
# Create the sensors table (pg 180)
    c.execute('''CREATE TABLE sensors(date text, city text,
    code text, sensor_id real, temperature real)''')
# Write a query to confirm your table was created – print the results
    for table in c.execute("SELECT name FROM sqlite_master WHERE type ='table'"):
                       print("Table", table[0])

#Insert random data into the table you created and then select all the results from the table (Select *)

    c.execute("INSERT INTO sensors VALUES('2016-11-05','Utrecht','Red',42,15.14)")
    c.execute('SELECT * FROM sensors')
    print(c.fetchone())
# Drop the table and confirm it has been dropped based on your previous query
con.execute("DROP TABLE sensors")
print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

#Close your connection
c.close()

Table sensors
('2016-11-05', 'Utrecht', 'Red', 42.0, 15.14)
# of tables 0


In [66]:
with sqlite3.connect(":memory:") as con:
    c = con.cursor()

# Create a list of tuples to form the pandas DataFrame
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = [tuple(x) for x in df.values]

#Create a table without specifying data types
con.execute('CREATE TABLE sunspots(year, sunactivity)')

# Insert records from a list of tuples – insert all the rows into the table and show the row count
con.executemany("INSERT INTO sunspots(year, sunactivity) VALUES (?, ?)", rows)
c.execute("SELECT COUNT(*) FROM sunspots")
print(c.fetchone())

#Delete the records where the sunactivity is greater than 20
print("Deleted", con.execute("DELETE FROM sunspots where sunactivity > 20").rowcount, "rows")

# Write a query to select all data where the year is less than 1732
print(read_sql("SELECT * FROM sunspots where year < 1732", con))

(309,)
Deleted 217 rows
      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1707.0         20.0
4   1708.0         10.0
5   1709.0          8.0
6   1710.0          3.0
7   1711.0          0.0
8   1712.0          0.0
9   1713.0          2.0
10  1714.0         11.0
11  1723.0         11.0


In [67]:
!pip3 install sqlalchemy

In [93]:
#Define the superclass
from sqlalchemy.orm import declarative_base

Base = declarative_base()

# Create the DBSession
Base.metadata.bind = engine

DBSession = sessionmaker(bind = engine)
session = DBSession()

#Create 2 stations and commit the session

session.add(Station(name='De Bilt'))
session.add(Station(name='Utrecht'))

session.commit()
print("Station", de_bilt)
# I could not fix the (sqlite3.IntegrityError) UNIQUE constraint failed

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: station.name
[SQL: INSERT INTO station (name) VALUES (?) RETURNING id]
[parameters: ('De Bilt',)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [80]:
# Insert a sensor record
temp_sensor = Sensor(last=20, multiplier=.1, station=de_bilt)
session.add(temp_sensor)
session.rollback()
session.commit()
print("Sensor", temp_sensor)


Sensor Id=%d last=%d multiplier=%.1f station_id=%d


C:\Users\bharo\AppData\Local\Temp\ipykernel_98280\721502804.py:4: SAWarning: Session's state has been changed on a non-active transaction - this state will be discarded.
  session.rollback()


In [91]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from pandas.io.sql import read_sql

# Create an engine from a URI
engine = create_engine('sqlite:///demo.db')
Base.metadata.create_all(engine)

#Create the station and sensor tables with the engine we just created
station = session.query(Station).first()
print ('First row in the station table : ', station)

#Select all the stations
print("Query 1", session.query(Station).all())

#Select all the sensors
print("Query 2", session.query(Sensor).all())

#Select the first sensor, which belongs to the first station
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())

#Query the station table with pandas read_sql()
print(read_sql("SELECT * FROM station", engine.raw_connection()))


First row in the station table :  Id=1 name=De Bilt
Query 1 [Id=1 name=De Bilt, Id=2 name=Utrecht]
Query 2 [Id=%d last=%d multiplier=%.1f station_id=%d]
Query 3 Id=%d last=%d multiplier=%.1f station_id=%d
   id     name
0   1  De Bilt
1   2  Utrecht


C:\Users\bharo\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
